In [2]:
! pip install wget

import os
import wget

url = 'http://rgai1.inf.u-szeged.hu/~berend/compsem/12lang_cbow_1000_0.1_bison-freq-sparse0cos_500000_0.75_pos.vec_sample.gz'
embedding_file_name = url.split('/')[-1]
if not os.path.exists(embedding_file_name):
    filename = wget.download(url)
    print(filename, " got downloaded")

  Using cached wget-3.2.zip (10 kB)
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9679 sha256=c5112d5863e3a9868dbf37bec5e452c9033eeb6aedbf3939ca9e582a5f8e4021
  Stored in directory: /home/berend/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
12lang_cbow_1000_0.1_bison-freq-sparse0cos_500000_0.75_pos.vec_sample.gz  got downloaded


The sparse word representations we just downloaded is compatible with [this test bed](https://github.com/wammar/multilingual-embeddings-eval-portal) introduced in [this paper](https://arxiv.org/abs/1602.01925).

In [3]:
import gzip
from collections import defaultdict

languages = ['de', 'en', 'es', 'hu', 'it']

per_language_words = defaultdict(list)
per_language_vectors = defaultdict(list)

def process_line(line):
    word, *rest = line.split()
    return word[3:], {f:float(v) for f,v in enumerate(rest) if float(v) > 0}

for l in gzip.open(embedding_file_name):
    line = l.decode('utf-8')
    lang_code = line[0:2]
    if lang_code in languages:
        word, features = process_line(line)
        per_language_words[lang_code].append(word)
        per_language_vectors[lang_code].append(features)
    elif len(per_language_words) == len(languages):
        break
per_language_w2i = {l:{w:i for i, w in enumerate(words)} for l, words in per_language_words.items()}

In [4]:
dog_vector = per_language_vectors['en'][per_language_w2i['en']['dog']]
perro_vector = per_language_vectors['es'][per_language_w2i['es']['perro']]
hund_vector = per_language_vectors['de'][per_language_w2i['de']['hund']]
kutya_vector = per_language_vectors['hu'][per_language_w2i['hu']['kutya']]
print("dog vector: ", dog_vector)
print("hund vector: ", hund_vector)
print("perro vector: ", perro_vector)
print("kutya vector: ", kutya_vector)

dog vector:  {168: 0.03209673, 209: 0.19435452, 231: 0.12448279, 356: 0.03813258, 383: 0.06006302, 389: 0.09654792, 404: 0.17842725, 563: 0.07398947, 596: 0.01282858, 673: 0.07237242, 681: 0.20265272, 742: 0.02527022, 856: 0.22130447, 916: 0.0377373}
hund vector:  {4: 0.05985065, 25: 0.01462543, 209: 0.05489186, 214: 0.15832504, 231: 0.07842155, 235: 0.00624106, 245: 0.00048997, 356: 0.1831656, 383: 0.19366023, 404: 0.06468459, 417: 0.06984168, 431: 0.02184164, 504: 0.06454777, 506: 0.04826535, 522: 0.10213534, 660: 0.01507871, 673: 0.09573983, 839: 0.1820208, 947: 0.07672509}
perro vector:  {4: 0.11355251, 99: 0.0071191, 152: 0.03780249, 169: 0.05487032, 231: 0.05204135, 286: 0.02734292, 383: 0.02551913, 384: 0.00730961, 400: 0.04727387, 404: 0.08808552, 417: 0.02315312, 551: 0.09886206, 563: 0.29823127, 597: 0.00438827, 628: 0.02810699, 660: 0.00971325, 673: 0.03192276, 722: 0.11359726, 756: 0.0180614, 766: 0.04156747, 768: 0.02204998, 836: 0.06824544, 853: 8.44e-06, 856: 0.09857655,

[Jaccard index](https://en.wikipedia.org/wiki/Jaccard_index) is a very simple but effective way to measure similartiy between two objects. Let's try to calculate it between two vectors $u$ and $v$.

In [4]:
def calculate_jaccard(u, v):
    # calculate the Jaccard similarity between two sets of features.
    pass